In [58]:
import os
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
#from torchvision.datasets import MNIST
#import torchvision.transforms as transforms
from torch.utils.data import Dataset
import spacy
#import pytorch_lightning as pl
#from pytorch_lightning import Trainer
import matplotlib.pyplot as plt
import numpy as np
from scipy import linalg
import scipy
from sklearn.manifold import TSNE
import sklearn
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.svm import LinearSVC, SVR
from sklearn.multioutput import MultiOutputRegressor
#from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
import scipy.io
import random
import inlp
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from typing import Tuple, Dict
from typing import List
import tqdm
import mnist_reader
import urllib
import time
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
def load_wiki(url = "http://nlp.biu.ac.il/~ravfogs/resources/wiki.1million.raw.txt"):
    
  response = urllib.request.urlopen(url)
  raw = response.read().decode('utf8')
  sents = raw.split("\n")
  
  return sents[:]


def create_docs(sentences: List[str]) -> List[spacy.tokens.Doc]:

    #parsing and toekenizing (NOTE: takes about 10 minutes)
    
    nlp = spacy.load('en_core_web_sm')
    nlp.remove_pipe("ner")
    
    start = time.time()
    
    docs = list(nlp.pipe(sentences, batch_size = 100))
    
    docs = list(docs)
    print(time.time() - start)
    
    return docs


def create_dataset(docs, num_samples = 100*1e3):
    
    data = []
    pbar = tqdm.tqdm(total = num_samples)
    
    while len(data) < num_samples:
        
        doc = random.choice(docs)
        sent = " ".join([tok.text for tok in doc])
        if len(doc) < 9: continue
        
        for i,tok in enumerate(doc):
            pbar.update(2)
            head = tok.head
            children = list(tok.children)
            related = [head] + children
            random_related = random.choice(related)
            data.append((sent, i, random_related.i, 1))
            
            # choose negative example
            
            chosen = False
            while not chosen:
                
                random_word = random.choice(doc)
                if random_word not in related:
                    chosen = True
                    data.append((sent, i, random_word.i, 0))
    
    random.shuffle(data)
    return data
                
        

## Collect data

In [3]:
sents = load_wiki()

In [4]:
sents = sents[:25000]
docs = create_docs(sents)

33.29659080505371


In [32]:
data = create_dataset(docs, num_samples = 5 * 1e4)





  0%|          | 0/50000.0 [00:00<?, ?it/s]



50012it [00:00, 290891.35it/s]                             


In [51]:
data[15]

("for instance , ` ` male pregnancy ' ' has been used to comedic effect in mainstream literature and films such as junior ( 1994 film , dir .",
 26,
 16,
 0)

## Encoder

In [89]:
class BertEncoder(object):
    
    def __init__(self, device = 'cpu'):

        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')
            
        self.model.eval()
        self.model.to(device)
        self.device = device
        
    def tokenize(self, original_sentence: List[str]) -> Tuple[List[str], Dict[int, int]]:

        """
        Parameters
        ----------
        Returns
        -------
        bert_tokens: The sentence, tokenized by BERT tokenizer.
        orig_to_tok_map: An output dictionary consisting of a mapping (alignment) between indices in the original tokenized sentence, and indices in the sentence tokenized by the BERT tokenizer. See https://github.com/google-research/bert
        """

        bert_tokens = ["[CLS]"]
        orig_to_tok_map = {}
        has_subwords = False
        is_subword = []

        for i, w in enumerate(original_sentence):
            tokenized_w = self.tokenizer.tokenize(w)
            has_subwords = len(tokenized_w) > 1
            is_subword.append(has_subwords)
            bert_tokens.extend(tokenized_w)

            orig_to_tok_map[i] = len(bert_tokens) - 1

        bert_tokens.append("[SEP]")
        
        return (bert_tokens, orig_to_tok_map)

    
    def encode(self, sentence: str):
        
        tokenized_text, orig2tok = self.tokenize(sentence.split(" "))
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
        tokens_tensor = torch.tensor([indexed_tokens]).to(self.device)

        with torch.no_grad():
            outputs = self.model(tokens_tensor)
            predictions = outputs[0].detach().cpu().numpy()
            return (predictions.squeeze(), orig2tok)
        

def collect_bert_states(bert, data: List[Tuple]):
    
    
    for i,d in tqdm.tqdm(enumerate(data), total = len(data)):
        
        sent = d[0]
        
        j,k = d[1], d[2]
        label = d[3]
        
        preds, orig2tok = bert.encode(sent)
        x1,x2 = preds[orig2tok[j]], preds[orig2tok[k]]
        data[i] = (sent, j, k, x1, x2, label)
    
    return data
        

In [90]:
bert = BertEncoder(device = "cpu")

In [94]:
data_with_states = collect_bert_states(bert, data[:10000])






  0%|          | 0/10000 [00:00<?, ?it/s]




  0%|          | 2/10000 [00:00<11:33, 14.42it/s]




  0%|          | 4/10000 [00:00<10:45, 15.48it/s]




  0%|          | 6/10000 [00:00<10:35, 15.72it/s]




  0%|          | 8/10000 [00:00<09:56, 16.76it/s]




  0%|          | 10/10000 [00:00<09:36, 17.32it/s]




  0%|          | 12/10000 [00:00<09:17, 17.91it/s]




  0%|          | 14/10000 [00:00<09:30, 17.52it/s]




  0%|          | 16/10000 [00:00<09:13, 18.03it/s]




  0%|          | 19/10000 [00:01<09:14, 17.99it/s]




  0%|          | 21/10000 [00:01<10:15, 16.23it/s]




  0%|          | 23/10000 [00:01<09:41, 17.17it/s]




  0%|          | 25/10000 [00:01<09:22, 17.72it/s]




  0%|          | 27/10000 [00:01<10:57, 15.16it/s]




  0%|          | 29/10000 [00:01<11:44, 14.15it/s]




  0%|          | 31/10000 [00:01<11:25, 14.55it/s]




  0%|          | 33/10000 [00:02<12:03, 13.79it/s]




  0%|          | 35/10000 [00:02<12:05, 13.74it/s]




  0%|          | 3

  3%|▎         | 276/10000 [00:15<08:56, 18.14it/s]




  3%|▎         | 278/10000 [00:15<10:01, 16.15it/s]




  3%|▎         | 280/10000 [00:15<09:44, 16.62it/s]




  3%|▎         | 282/10000 [00:15<09:19, 17.37it/s]




  3%|▎         | 284/10000 [00:15<08:58, 18.04it/s]




  3%|▎         | 286/10000 [00:15<08:52, 18.24it/s]




  3%|▎         | 289/10000 [00:15<08:32, 18.96it/s]




  3%|▎         | 291/10000 [00:15<08:53, 18.21it/s]




  3%|▎         | 293/10000 [00:16<08:40, 18.66it/s]




  3%|▎         | 295/10000 [00:16<09:44, 16.60it/s]




  3%|▎         | 297/10000 [00:16<10:04, 16.06it/s]




  3%|▎         | 299/10000 [00:16<09:36, 16.83it/s]




  3%|▎         | 302/10000 [00:16<08:55, 18.12it/s]




  3%|▎         | 305/10000 [00:16<08:26, 19.14it/s]




  3%|▎         | 307/10000 [00:16<08:31, 18.96it/s]




  3%|▎         | 309/10000 [00:16<08:57, 18.02it/s]




  3%|▎         | 311/10000 [00:17<08:46, 18.41it/s]




  3%|▎         | 313/10000 [00:17<08:39, 18.66it

  6%|▌         | 552/10000 [00:30<11:26, 13.76it/s]




  6%|▌         | 554/10000 [00:30<10:42, 14.69it/s]




  6%|▌         | 557/10000 [00:30<09:50, 15.99it/s]




  6%|▌         | 560/10000 [00:30<09:12, 17.09it/s]




  6%|▌         | 562/10000 [00:31<09:26, 16.66it/s]




  6%|▌         | 564/10000 [00:31<08:59, 17.48it/s]




  6%|▌         | 566/10000 [00:31<08:55, 17.61it/s]




  6%|▌         | 568/10000 [00:31<09:10, 17.14it/s]




  6%|▌         | 571/10000 [00:31<08:34, 18.34it/s]




  6%|▌         | 574/10000 [00:31<08:21, 18.79it/s]




  6%|▌         | 576/10000 [00:31<08:37, 18.22it/s]




  6%|▌         | 578/10000 [00:31<08:33, 18.37it/s]




  6%|▌         | 581/10000 [00:32<08:12, 19.14it/s]




  6%|▌         | 584/10000 [00:32<07:55, 19.81it/s]




  6%|▌         | 587/10000 [00:32<07:41, 20.38it/s]




  6%|▌         | 590/10000 [00:32<08:04, 19.42it/s]




  6%|▌         | 592/10000 [00:32<09:02, 17.34it/s]




  6%|▌         | 594/10000 [00:32<09:02, 17.33it

  9%|▊         | 851/10000 [00:46<07:54, 19.29it/s]




  9%|▊         | 854/10000 [00:46<08:01, 18.99it/s]




  9%|▊         | 856/10000 [00:46<08:03, 18.91it/s]




  9%|▊         | 858/10000 [00:46<08:02, 18.93it/s]




  9%|▊         | 860/10000 [00:46<08:49, 17.27it/s]




  9%|▊         | 862/10000 [00:46<08:44, 17.41it/s]




  9%|▊         | 864/10000 [00:47<08:44, 17.43it/s]




  9%|▊         | 866/10000 [00:47<09:15, 16.45it/s]




  9%|▊         | 868/10000 [00:47<09:59, 15.22it/s]




  9%|▊         | 871/10000 [00:47<09:17, 16.39it/s]




  9%|▊         | 873/10000 [00:47<09:16, 16.39it/s]




  9%|▉         | 875/10000 [00:47<09:02, 16.82it/s]




  9%|▉         | 877/10000 [00:47<08:45, 17.35it/s]




  9%|▉         | 880/10000 [00:47<08:38, 17.60it/s]




  9%|▉         | 882/10000 [00:48<11:22, 13.36it/s]




  9%|▉         | 884/10000 [00:48<11:31, 13.19it/s]




  9%|▉         | 886/10000 [00:48<10:31, 14.44it/s]




  9%|▉         | 888/10000 [00:48<10:11, 14.91it

 11%|█         | 1098/10000 [01:01<09:36, 15.44it/s]




 11%|█         | 1100/10000 [01:01<09:51, 15.05it/s]




 11%|█         | 1102/10000 [01:01<09:22, 15.81it/s]




 11%|█         | 1104/10000 [01:02<09:27, 15.69it/s]




 11%|█         | 1106/10000 [01:02<09:33, 15.51it/s]




 11%|█         | 1109/10000 [01:02<09:20, 15.85it/s]




 11%|█         | 1111/10000 [01:02<09:32, 15.53it/s]




 11%|█         | 1113/10000 [01:02<09:06, 16.25it/s]




 11%|█         | 1115/10000 [01:02<09:09, 16.18it/s]




 11%|█         | 1117/10000 [01:02<08:56, 16.57it/s]




 11%|█         | 1119/10000 [01:02<08:47, 16.84it/s]




 11%|█         | 1121/10000 [01:03<08:24, 17.60it/s]




 11%|█         | 1123/10000 [01:03<08:27, 17.49it/s]




 11%|█▏        | 1125/10000 [01:03<08:40, 17.05it/s]




 11%|█▏        | 1127/10000 [01:03<08:24, 17.60it/s]




 11%|█▏        | 1129/10000 [01:03<08:07, 18.21it/s]




 11%|█▏        | 1131/10000 [01:03<08:35, 17.20it/s]




 11%|█▏        | 1134/10000 [01

...

 86%|████████▋ | 8626/10000 [08:14<01:19, 17.29it/s]




 86%|████████▋ | 8629/10000 [08:14<01:13, 18.77it/s]




 86%|████████▋ | 8631/10000 [08:14<01:13, 18.64it/s]




 86%|████████▋ | 8633/10000 [08:15<01:12, 18.98it/s]




 86%|████████▋ | 8636/10000 [08:15<01:08, 19.92it/s]




 86%|████████▋ | 8639/10000 [08:15<01:09, 19.50it/s]




 86%|████████▋ | 8642/10000 [08:15<01:09, 19.41it/s]




 86%|████████▋ | 8645/10000 [08:15<01:06, 20.29it/s]




 86%|████████▋ | 8648/10000 [08:15<01:06, 20.22it/s]




 87%|████████▋ | 8651/10000 [08:15<01:04, 20.89it/s]




 87%|████████▋ | 8654/10000 [08:16<01:10, 19.04it/s]




 87%|████████▋ | 8657/10000 [08:16<01:08, 19.50it/s]




 87%|████████▋ | 8660/10000 [08:16<01:08, 19.58it/s]




 87%|████████▋ | 8662/10000 [08:16<01:15, 17.66it/s]




 87%|████████▋ | 8665/10000 [08:16<01:11, 18.69it/s]




 87%|████████▋ | 8668/10000 [08:16<01:09, 19.25it/s]




 87%|████████▋ | 8670/10000 [08:16<01:08, 19.29it/s]




 87%|████████▋ | 8673/10000 [08

 89%|████████▉ | 8897/10000 [08:29<01:00, 18.25it/s]




 89%|████████▉ | 8900/10000 [08:29<01:00, 18.08it/s]




 89%|████████▉ | 8902/10000 [08:29<00:59, 18.35it/s]




 89%|████████▉ | 8904/10000 [08:30<01:00, 18.15it/s]




 89%|████████▉ | 8906/10000 [08:30<00:59, 18.32it/s]




 89%|████████▉ | 8908/10000 [08:30<01:00, 17.95it/s]




 89%|████████▉ | 8910/10000 [08:30<01:01, 17.82it/s]




 89%|████████▉ | 8912/10000 [08:30<01:01, 17.83it/s]




 89%|████████▉ | 8914/10000 [08:30<00:59, 18.20it/s]




 89%|████████▉ | 8917/10000 [08:30<00:55, 19.46it/s]




 89%|████████▉ | 8919/10000 [08:30<00:57, 18.69it/s]




 89%|████████▉ | 8922/10000 [08:30<00:55, 19.49it/s]




 89%|████████▉ | 8925/10000 [08:31<00:52, 20.30it/s]




 89%|████████▉ | 8928/10000 [08:31<00:51, 20.62it/s]




 89%|████████▉ | 8931/10000 [08:31<00:52, 20.54it/s]




 89%|████████▉ | 8934/10000 [08:31<00:49, 21.37it/s]




 89%|████████▉ | 8937/10000 [08:31<00:52, 20.19it/s]




 89%|████████▉ | 8940/10000 [08

 92%|█████████▏| 9171/10000 [08:44<00:49, 16.58it/s]




 92%|█████████▏| 9174/10000 [08:44<00:47, 17.28it/s]




 92%|█████████▏| 9176/10000 [08:44<00:48, 16.92it/s]




 92%|█████████▏| 9178/10000 [08:44<00:50, 16.16it/s]




 92%|█████████▏| 9180/10000 [08:44<00:48, 17.04it/s]




 92%|█████████▏| 9182/10000 [08:45<00:48, 16.77it/s]




 92%|█████████▏| 9184/10000 [08:45<00:47, 17.35it/s]




 92%|█████████▏| 9186/10000 [08:45<00:45, 17.78it/s]




 92%|█████████▏| 9188/10000 [08:45<00:48, 16.63it/s]




 92%|█████████▏| 9190/10000 [08:45<00:49, 16.43it/s]




 92%|█████████▏| 9192/10000 [08:45<00:46, 17.33it/s]




 92%|█████████▏| 9195/10000 [08:45<00:43, 18.62it/s]




 92%|█████████▏| 9197/10000 [08:45<00:47, 16.82it/s]




 92%|█████████▏| 9199/10000 [08:46<00:51, 15.66it/s]




 92%|█████████▏| 9201/10000 [08:46<00:48, 16.41it/s]




 92%|█████████▏| 9203/10000 [08:46<00:48, 16.59it/s]




 92%|█████████▏| 9206/10000 [08:46<00:45, 17.42it/s]




 92%|█████████▏| 9209/10000 [08

 94%|█████████▍| 9439/10000 [08:59<00:33, 16.56it/s]




 94%|█████████▍| 9441/10000 [08:59<00:34, 16.33it/s]




 94%|█████████▍| 9444/10000 [08:59<00:31, 17.67it/s]




 94%|█████████▍| 9446/10000 [08:59<00:30, 18.09it/s]




 94%|█████████▍| 9449/10000 [09:00<00:28, 19.06it/s]




 95%|█████████▍| 9451/10000 [09:00<00:29, 18.58it/s]




 95%|█████████▍| 9454/10000 [09:00<00:28, 19.15it/s]




 95%|█████████▍| 9457/10000 [09:00<00:27, 19.86it/s]




 95%|█████████▍| 9460/10000 [09:00<00:27, 19.91it/s]




 95%|█████████▍| 9463/10000 [09:00<00:27, 19.44it/s]




 95%|█████████▍| 9466/10000 [09:00<00:25, 20.55it/s]




 95%|█████████▍| 9469/10000 [09:00<00:25, 20.45it/s]




 95%|█████████▍| 9472/10000 [09:01<00:25, 20.57it/s]




 95%|█████████▍| 9475/10000 [09:01<00:26, 19.89it/s]




 95%|█████████▍| 9478/10000 [09:01<00:28, 18.55it/s]




 95%|█████████▍| 9481/10000 [09:01<00:27, 19.03it/s]




 95%|█████████▍| 9484/10000 [09:01<00:26, 19.75it/s]




 95%|█████████▍| 9486/10000 [09

 97%|█████████▋| 9731/10000 [09:14<00:13, 19.27it/s]




 97%|█████████▋| 9734/10000 [09:15<00:13, 19.30it/s]




 97%|█████████▋| 9736/10000 [09:15<00:13, 19.31it/s]




 97%|█████████▋| 9739/10000 [09:15<00:12, 20.99it/s]




 97%|█████████▋| 9742/10000 [09:15<00:13, 19.75it/s]




 97%|█████████▋| 9745/10000 [09:15<00:12, 20.13it/s]




 97%|█████████▋| 9748/10000 [09:15<00:12, 19.41it/s]




 98%|█████████▊| 9751/10000 [09:15<00:11, 20.92it/s]




 98%|█████████▊| 9754/10000 [09:16<00:12, 19.17it/s]




 98%|█████████▊| 9756/10000 [09:16<00:15, 15.76it/s]




 98%|█████████▊| 9758/10000 [09:16<00:15, 16.03it/s]




 98%|█████████▊| 9761/10000 [09:16<00:13, 17.47it/s]




 98%|█████████▊| 9763/10000 [09:16<00:14, 16.89it/s]




 98%|█████████▊| 9766/10000 [09:16<00:12, 18.18it/s]




 98%|█████████▊| 9768/10000 [09:16<00:14, 15.99it/s]




 98%|█████████▊| 9771/10000 [09:17<00:13, 17.42it/s]




 98%|█████████▊| 9773/10000 [09:17<00:12, 18.10it/s]




 98%|█████████▊| 9775/10000 [09

In [ ]:
random.shuffle(data_with_states)

In [93]:
import pickle
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
import loss
import scipy
from scipy import linalg


class Dataset(torch.utils.data.Dataset):
    
    """Simple torch dataset class"""
    def __init__(self, data, device):

        self.data = data
        self.device = device

    def __len__(self):

        return len(self.data)

    def __getitem__(self, index):

        with torch.no_grad():
             
            vec1_np, vec2_np, str1, str2, _ = self.data[index]
            
            vec1, vec2, str1, str2, pair_id = self.data[index]            
            vec1, vec2 = torch.from_numpy(vec1_np).float(), torch.from_numpy(vec2_np).float()
            vec1 = vec1.to(self.device)
            vec2 = vec2.to(self.device)
            
            return (vec1, vec2, str1, str2, pair_id)
            
            
            
class Siamese(pl.LightningModule):

    def __init__(self, X_train, X_dev, dim, batch_size, dropout_rate, device = "cuda"):
        super(Siamese, self).__init__()
        self.l = torch.nn.Linear(1024, dim)
        
        self.train_data = Dataset(X_train, device)
        self.dev_data = Dataset(X_dev, device)
        self.train_gen = torch.utils.data.DataLoader(self.train_data, batch_size = batch_size, drop_last = False, shuffle=True)
        self.dev_gen = torch.utils.data.DataLoader(self.dev_data, batch_size = batch_size, drop_last = False, shuffle=True)
        self.loss_fn = loss.BatchHardTripletLoss(final = "softmax", device = device)
        self.dropout = torch.nn.Dropout(p = dropout_rate)
        self.acc = None
        self.optimizer = torch.optim.Adam(self.parameters(), weight_decay = 1e-6)
        
    def forward(self, x1, x2):

          h1 = self.l(x1)
          h2 = self.l(x2)
         
          return h1, h2
 
    def train_network(self, num_epochs):
    
      trainer = Trainer(max_nb_epochs = num_epochs, min_nb_epochs = num_epochs, show_progress_bar = False)
      trainer.fit(self)

      return self.acc   
      
    def get_weights(self):
    
        return self.l.weight.data.detach().cpu().numpy()
    
    def training_step(self, batch, batch_nb):
        # REQUIRED
        x1, x2, str1, str2, ids = batch
        h1, h2 = self.forward(self.dropout(x1), self.dropout(x2))
        loss_val =  self.loss_fn(h1, h2, str1, str2, ids, index=0, evaluation = False)
        
        return {'loss': loss_val[0]}
        

    def validation_step(self, batch, batch_nb):
    
        # OPTIONAL
        x1, x2, str1, str2, ids = batch
        h1, h2 = self.forward(x1, x2)
        loss_val =  self.loss_fn(h1, h2, str1, str2, ids, index=batch_nb, evaluation = True)
        return {'val_loss': loss_val[0]}

    def validation_end(self, outputs):
        # OPTIONAL    
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print("Loss is {}".format(avg_loss))
        return {'avg_val_loss': avg_loss}

    def configure_optimizers(self):
        # REQUIRED
        return torch.optim.Adam(self.parameters(), weight_decay = 1e-4)

    @pl.data_loader
    def train_dataloader(self):
        return self.train_gen

    @pl.data_loader
    def val_dataloader(self):
        # OPTIONAL
        # can also return a list of val dataloaders
        return self.dev_gen

('by 2009 it had become a tourist attraction , but hackney council had wanted to remove all graffiti from the area and tried to contact the building owner to gain her agreement to remove the artwork .',
 20,
 20,
 array([ 6.12367094e-01,  3.23530257e-01,  1.57817781e-01, -2.75148302e-01,
         7.72554755e-01, -2.12912690e-02,  1.69373408e-01,  9.08584952e-01,
         4.40908074e-01,  1.10624366e-01, -3.86447161e-02, -8.46915483e-01,
        -4.56419170e-01,  4.98859107e-01, -4.86403137e-01,  1.67331085e-01,
         2.24158064e-01, -2.20201179e-01,  1.02436133e-02,  8.29810083e-01,
        -2.79543791e-02,  3.48104984e-01, -1.10657501e+00,  1.75714597e-01,
         5.20247877e-01, -9.77819264e-02,  1.89434260e-01, -5.66880822e-01,
         1.37920126e-01, -2.21476108e-01,  4.81060773e-01,  1.91016629e-01,
        -2.88962454e-01, -4.64747488e-01,  3.69946480e-01,  4.56393361e-01,
         1.83483034e-01,  1.52081490e-01, -7.23235667e-01,  1.47964388e-01,
        -2.98851073e-01, -3